Production planning problem from book Operations Research - A Model-Baesed Approach 3rd Edition by H.A Eiselt and Carl-Louis Sandblom



In [19]:
import polars as pl

processing_times = pl.read_csv("processing_times.csv")
machines_availability = pl.read_csv('machines_availability.csv')
products_profit = pl.read_csv("products_profit.csv")

In [26]:
import pyoframe as pf

m = pf.Model('production_planning_pyoframe', solver='gurobi')

m.Production = pf.Variable(products_profit[['products']], lb=0)

m.Production.data


products,__variable_id
i64,u32
1,1
2,2
3,3


In [27]:
machine_usage = m.Production * processing_times
machine_usage

<Expression size=6 dimensions={'products': 3, 'machines': 2} terms=6>
[1,1]: 3 Production[1]
[2,1]: 5 Production[2]
[3,1]: 4 Production[3]
[1,2]: 6 Production[1]
[2,2]: Production[2]
[3,2]: 3 Production[3]

In [28]:

m.machine_capacity = pf.sum_by('machines', machine_usage) <= machines_availability
m.machine_capacity.data

machines,__constraint_id
i64,u32
1,0
2,1


In [23]:
total_profit_per_product = products_profit * m.Production
total_profit_per_product

<Expression size=3 dimensions={'products': 3} terms=3>
[1]: 5 Production[1]
[2]: 3.5 Production[2]
[3]: 4.5 Production[3]

In [24]:
m.maximize = pf.sum(total_profit_per_product)
m.objective

<Objective size=1 dimensions={} terms=3>
objective: 5 Production[1] +3.5 Production[2] +4.5 Production[3]

In [25]:
m.optimize()
m.Production.solution

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.4.0 24E248)
Gurobi Compute Server Worker version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x16838762
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+02   8.745950e+01   0.000000e+00      0s
       2    6.4000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.400000000e+02


products,solution
i64,f64
1,20.0
2,0.0
3,120.0


In [9]:
import pandas as pd

pt: pd.DataFrame = processing_times.to_pandas()
from rustpy_xlsxwriter import save_records

pt_dict = pt.to_dict(orient='records')
save_records(pt_dict, 'processing_times.xlsx')
